In [1]:
!pip install Flask pyngrok

In [2]:
!ngrok authtoken 2ml1tvUfzrhhV7fLlwn1ixZUxsy_5ijjeyeRm2hJcqAL3M4XH

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:
import cv2
import numpy as np
import os
from flask import Flask, request, jsonify, send_from_directory, render_template_string
from pyngrok import ngrok

app = Flask(__name__)

# Define the output folder and ensure it exists
OUTPUT_FOLDER = '/content/drive/MyDrive/uploads'
os.makedirs(OUTPUT_FOLDER, exist_ok=True)

def bluish_grey_to_black(image_path, output_path):
    image = cv2.imread(image_path)
    hsv_image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    lower_blue = np.array([0, 0, 50])
    upper_blue = np.array([180, 50, 200])
    mask = cv2.inRange(hsv_image, lower_blue, upper_blue)
    image[mask > 0] = [0, 0, 0]
    cv2.imwrite(output_path, image)
    return image

def calculate_leaf_area(image):
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    total_area = 256 * 256
    black_area = np.sum(gray_image == 0)
    leaf_area = total_area - black_area
    return leaf_area

def read_yolo_instance_segmentation_file(file_path):
    with open(file_path, 'r') as file:
        lines = file.readlines()
    annotations = []
    for line in lines:
        parts = line.strip().split()
        class_id = int(parts[0])
        points = [(float(parts[i]) * 256, float(parts[i + 1]) * 256) for i in range(1, len(parts), 2)]
        annotations.append((class_id, points))
    return annotations

def polygon_area(points):
    n = len(points)
    if n < 3:
        return 0
    area = 0.0
    for i in range(n):
        x1, y1 = points[i]
        x2, y2 = points[(i + 1) % n]
        area += x1 * y2
        area -= x2 * y1
    area = abs(area) / 2.0
    return area

def categorize_area(total_area,affected_area):
    if total_area == 0:
        return "Total area cannot be zero."
    percentage = (affected_area / total_area) * 100
    if percentage < 0:
        return "Invalid affected area."
    elif percentage <= 4:
        category = "Normal"
    elif percentage <= 20:
        category = "Infected"
    elif percentage <= 40:
        category = "Serious"
    elif percentage <= 60:
        category = "Critical"
    elif percentage <= 80:
        category = "Very Critical"
    else:
        category = "Dead"
    return percentage, category

@app.route('/')
def index():
    # Frontend HTML with result display
    html = '''
    <!DOCTYPE html>
    <html lang="en">
    <head>
        <meta charset="UTF-8">
        <meta name="viewport" content="width=device-width, initial-scale=1.0">
        <title>Leaf Area Detection</title>
        <style>
            body {
                font-family: Arial, sans-serif;
                margin: 20px;
            }
            h1 {
                color: #333;
            }
            #result {
                margin-top: 20px;
                padding: 10px;
                border: 1px solid #ccc;
                display: none; /* Initially hidden */
            }
        </style>
    </head>
    <body>
        <h1>Upload Image for Leaf Area Processing</h1>
        <form id="uploadForm" action="/process_image" method="post" enctype="multipart/form-data">
            <input type="file" name="image" accept="image/*" required>
            <button type="submit">Upload</button>
        </form>
        <div id="result">
            <h2>Processing Result</h2>
            <p id="leafArea"></p>
            <p id="totalArea"></p>
            <p id="category"></p>
        </div>
        <script>
            const form = document.getElementById('uploadForm');
            form.onsubmit = async (e) => {
                e.preventDefault();
                const formData = new FormData(form);
                const response = await fetch(form.action, {
                    method: 'POST',
                    body: formData
                });
                const result = await response.json();
                if (response.ok) {
                    document.getElementById('leafArea').innerText = "Leaf Area: " + result.leaf_area + " pixels";
                    document.getElementById('totalArea').innerText = "Total affected9 Area: " + result.total_area + " pixels";
                    document.getElementById('category').innerText = "Category: " + result.category;
                    document.getElementById('result').style.display = 'block'; // Show result
                } else {
                    alert("Error: " + result.error);
                }
            };
        </script>
    </body>
    </html>
    '''
    return render_template_string(html)

@app.route('/process_image', methods=['POST'])
def process_image():
    if 'image' not in request.files:
        return jsonify({"error": "No file part"}), 400
    file = request.files['image']
    if file.filename == '':
        return jsonify({"error": "No selected file"}), 400

    # Save the uploaded file
    image_path = os.path.join(OUTPUT_FOLDER, file.filename)
    output_path = os.path.join(OUTPUT_FOLDER, f'processed_{file.filename}')
    print(f"Saving uploaded image to: {image_path}")
    file.save(image_path)

    try:
        # Process the image
        processed_image = bluish_grey_to_black(image_path, output_path)
        leaf_area = calculate_leaf_area(processed_image)
        print(f"Leaf area calculated: {leaf_area}")

        # path to text dir where your segmented images along with the labels is saved
        txt_dir = '/content/drive/MyDrive/predict/meow611/labels'
        total_area = 0.0

        for filename in os.listdir(txt_dir):
            if filename.endswith('.txt'):
                file_path = os.path.join(txt_dir, filename)
                annotations = read_yolo_instance_segmentation_file(file_path)
                for class_id, points in annotations:
                    area = polygon_area(points)
                    total_area += area

        percentage, category = categorize_area(leaf_area, total_area)

        # Convert to standard Python types for JSON serialization
        return jsonify({
            "leaf_area": int(leaf_area),  # Ensure it's a standard Python int
            "total_area": float(total_area),  # Ensure it's a standard Python float
            "category": category
        })

    except Exception as e:
        return jsonify({"error": str(e)}), 500

@app.route('/process_image/<path:filename>')
def serve_output_image(filename):
    return send_from_directory(OUTPUT_FOLDER, filename)

if __name__ == "__main__":
    # Expose the Flask app on port 5000
    public_url = ngrok.connect(5000)
    print(f"Your app is running at: {public_url}")

    # Start the Flask app
    app.run(port=5000, debug=False)


Your app is running at: NgrokTunnel: "https://c176-35-185-62-207.ngrok-free.app" -> "http://localhost:5000"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [02/Oct/2024 19:27:10] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [02/Oct/2024 19:27:10] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [02/Oct/2024 19:27:19] "POST /process_image HTTP/1.1" 200 -


Saving uploaded image to: /content/drive/MyDrive/uploads/soni.JPG
Leaf area calculated: 31849


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive
